In [2]:
import numpy as np
import polars as pl
from pathlib import Path

In [4]:
TICKER = "WHIRLPOOL.NS"
TRAIN_FILE = Path("../datasets") / f"{TICKER}"

TRACKERS = {
    "correct_trade": 0,
    "wrong_trade": 0,
    "buy_counter": 0,
    "sell_counter": 0,
    "hold_counter": 0,
    "good_hold_counter": 0,
    "good_sell_counter": 0,
    "good_buy_counter": 0,
    "bad_buy_counter": 0,
    "bad_hold_counter": 0,
    "bad_sell_counter": 0,
    "holds_with_no_shares_counter": 0,
    "bad_holds_with_no_shares_counter": 0,
    "good_holds_with_no_shares_counter": 0,
    "good_hold_profit": 0,
    "good_sell_profit": 0,
    "bad_hold_loss": 0,
    "bad_sell_loss": 0,
    "good_profit": 0,
    "bad_loss": 0,
    "good_hold_streak": 0,
    "bad_hold_streak": 0,
    "waiting_streak": 0,
    "reward_tracker": 0,
    "available_amount": 10_000,
    "shares_holding": 0,
    "buy_price_index": -1,
    "buy_price": -1,
    "total_buy_price": -1,
    "profit": 0,
    "portfolio_value": 10_000,
    "portfolio_value_threshold": 10_000,
}


OBS = (
    pl
    .read_parquet(TRAIN_FILE)
    .with_columns(index=pl.int_range(0, end=pl.count(), eager=False))
    .sort("index")
    .set_sorted("index")
    .group_by_dynamic(
        "index", every="1i", period="40i", include_boundaries=True, closed="right"
    )
    .agg(pl.col("Close"), pl.col("Datetime").last())
    .with_columns(pl.col("Close").list.len().alias("counter"))
    .filter(pl.col("counter") == 40)
    .with_columns(index=pl.int_range(0, end=pl.count(), eager=False))
    .select("index", "Close", "Datetime")
    # .with_columns(
    #     close_price=pl.col("Close").list.last(),
    #     min_price=pl.col("Close").list.min(),
    #     max_price=pl.col("Close").list.max(),
    # )
    # .with_columns([
    #     ((pl.col("close_price") - pl.col("min_price")) / (pl.col("max_price") - pl.col("min_price"))*100).alias("%_away_from_min")
    # ])
    # .with_columns([
    #     (100 - pl.col("%_away_from_min")).alias("%_away_from_max")
    # ])
    # .with_columns([
    #     pl.lit(v).alias(k)
    #     for k, v in TRACKERS.items()
    # ])
)
OBS

index,Close,Datetime
i64,list[f64],"datetime[ns, UTC]"
0,"[2464.649902, 2481.449951, … 2519.0]",2021-01-19 07:45:00 UTC
1,"[2481.449951, 2520.550049, … 2511.149902]",2021-01-19 08:45:00 UTC
2,"[2520.550049, 2509.949951, … 2517.0]",2021-01-19 09:45:00 UTC
3,"[2509.949951, 2514.550049, … 2535.899902]",2021-01-20 03:45:00 UTC
4,"[2514.550049, 2508.899902, … 2533.850098]",2021-01-20 04:45:00 UTC
5,"[2508.899902, 2511.850098, … 2536.600098]",2021-01-20 05:45:00 UTC
6,"[2511.850098, 2485.0, … 2534.100098]",2021-01-20 06:45:00 UTC
7,"[2485.0, 2462.300049, … 2543.100098]",2021-01-20 07:45:00 UTC
8,"[2462.300049, 2459.699951, … 2537.300049]",2021-01-20 08:45:00 UTC


In [37]:
(
    OBS
    .select([
    "min_price", "close_price", "max_price",
    "%_away_from_min", "%_away_from_max"
    ])
)

min_price,close_price,max_price,%_away_from_min,%_away_from_max
f64,f64,f64,f64,f64
2369.0,2519.0,2520.550049,98.977203,1.022797
2369.0,2511.149902,2520.550049,93.797332,6.202668
2369.0,2517.0,2520.550049,97.657507,2.342493
2369.0,2535.899902,2535.899902,100.0,0.0
2369.0,2533.850098,2535.899902,98.771836,1.228164
2369.0,2536.600098,2536.600098,100.0,0.0
2369.0,2534.100098,2536.600098,98.508354,1.491646
2369.0,2543.100098,2543.100098,100.0,0.0
2369.0,2537.300049,2543.100098,96.668555,3.331445
